In [1]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=9c8f4858a77b755f634920a8d56714c80bd0d13ac7a70859b8bb1522e4df72f3
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 4.3 MB/s 
     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 6.6 MB 75.4 MB/s 
     |████████████████████████████████| 895 kB 80.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [23]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [24]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [25]:
df['text'] = df['text'].astype('str') 

In [8]:
#df_sample = df.sample(n=50000, random_state=2)

In [9]:
#df_sample

In [10]:
docs = list(df.loc[:, "text"].values)

In [11]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [12]:
178255 * 5

891275

In [13]:
docs = docs[713020:891275]

In [14]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [3:10:13<00:00, 15.62it/s]


In [26]:
df = df[713020:891275]

In [27]:
df

,text
713020,Why is cnn not in Ukraine covering the war. Re...
713021,"#Russianpeople, you can resolve this. Is killi..."
713022,@EssexPR It’s so not their fault. This is Puti...
713023,Russia's chief negotiator Vladimir Medinsky sa...
713024,It's no coincidence that as the West declares ...
...,...
891270,Russia-Ukraine war LIVE: Russian convoy advanc...
891271,115 Nigerian men offer to join Ukraine in figh...
891272,@WSJ STOP RUSSIA NOW‼️‼️‼️‼️\nHELP #Kharkiv NO...
891273,@ZelenskyyUa according global values india giv...


In [28]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [29]:
df

,text,Label,Negative,Neutral,Positive
713020,Why is cnn not in Ukraine covering the war. Re...,0,0.654918,0.323429,0.021653
713021,"#Russianpeople, you can resolve this. Is killi...",0,0.955211,0.040655,0.004134
713022,@EssexPR It’s so not their fault. This is Puti...,0,0.543219,0.414597,0.042184
713023,Russia's chief negotiator Vladimir Medinsky sa...,1,0.025684,0.963019,0.011296
713024,It's no coincidence that as the West declares ...,0,0.793615,0.201297,0.005088
...,...,...,...,...,...
891270,Russia-Ukraine war LIVE: Russian convoy advanc...,1,0.049781,0.942556,0.007663
891271,115 Nigerian men offer to join Ukraine in figh...,1,0.003173,0.931986,0.064841
891272,@WSJ STOP RUSSIA NOW‼️‼️‼️‼️\nHELP #Kharkiv NO...,0,0.771638,0.214663,0.013699
891273,@ZelenskyyUa according global values india giv...,1,0.030177,0.916394,0.053430


In [30]:
df

,text,Label,Negative,Neutral,Positive
713020,Why is cnn not in Ukraine covering the war. Re...,0,0.654918,0.323429,0.021653
713021,"#Russianpeople, you can resolve this. Is killi...",0,0.955211,0.040655,0.004134
713022,@EssexPR It’s so not their fault. This is Puti...,0,0.543219,0.414597,0.042184
713023,Russia's chief negotiator Vladimir Medinsky sa...,1,0.025684,0.963019,0.011296
713024,It's no coincidence that as the West declares ...,0,0.793615,0.201297,0.005088
...,...,...,...,...,...
891270,Russia-Ukraine war LIVE: Russian convoy advanc...,1,0.049781,0.942556,0.007663
891271,115 Nigerian men offer to join Ukraine in figh...,1,0.003173,0.931986,0.064841
891272,@WSJ STOP RUSSIA NOW‼️‼️‼️‼️\nHELP #Kharkiv NO...,0,0.771638,0.214663,0.013699
891273,@ZelenskyyUa according global values india giv...,1,0.030177,0.916394,0.053430


In [31]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p5.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p5.csv 'drive/My Drive/Colab Notebooks'

Mounted at drive
